In [1]:
%load_ext autoreload
%autoreload 2
import yaml
import pathlib
from os.path import join
import os
import sys
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio import SeqIO

current_path = str(pathlib.Path(os.getcwd()).parent)
sys.path.append(current_path)

#import sys
#sys.path.append('../)

from config_utils import *
from sequtils import *
from prokbert_tokenizer import ProkBERTTokenizer
    
pd.set_option('display.max_rows', 10000)
pd.set_option('display.max_columns', 30)
#pd.set_option('display.width', 4000)
#pd.set_option('display.max_colwidth', None)
pd.set_option('display.expand_frame_repr', True)
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"
%load_ext autoreload
%autoreload 2
defconfig = SeqConfig()
         
segmentation_params = {'type': 'random'}
tokenization_params = {'shift':2} 
                   
comp_params = defconfig.get_set_computational_paramters()
tokenizer = ProkBERTTokenizer(tokenization_params=tokenization_params, 
                              segmentation_params=segmentation_params,
                              operation_space='sequence')
segment = 'AATCAAGGAATTATTATCGTT'
       
segmentation_params = defconfig.get_set_segmentation_parameters()
tokenization_params = defconfig.get_and_set_tokenization_params({'shift':2})             
comp_params = defconfig.get_set_computational_paramters()

num_cores = comp_params['cpu_cores_for_tokenization']
batch_size = comp_params['batch_size_tokenization']
numpy_dtype=comp_params['np_tokentype'] # Note that if you use kmer>7 then other is recommended
input_fasta_dir = join(current_path, 'data/sample_data/pretraining')

input_fasta_files = [join(input_fasta_dir, file) for file in get_non_empty_files(input_fasta_dir)]
contigs = load_contigs(input_fasta_files,IsAddHeader=True,AsDataFrame=True, adding_reverse_complement=False)
contigs['sequence_id'] = list(range(len(contigs)))
contigs = contigs[['sequence', 'sequence_id']]
segment_db = segment_sequences(contigs, segmentation_params, AsDataFrame=True)

tokenized = batch_tokenize_segments_with_ids(segment_db, tokenization_params, num_cores, batch_size, numpy_dtype)
expected_max_token = max(len(arr) for arrays in tokenized.values() for arr in arrays)
X, torchdb = get_rectangular_array_from_tokenized_dataset(tokenized,
                                                          tokenization_params['shift'],
                                                          expected_max_token, numpy_dtype = numpy_dtype)
hdf_file = '../data/preprocessed/pretraining.h5'
save_to_hdf(X, hdf_file, database = torchdb, compression=True)
                  

2023-08-17 13:39:16,580 - INFO - Note: NumExpr detected 48 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2023-08-17 13:39:16,581 - INFO - NumExpr defaulting to 8 threads.


ImportError: attempted relative import with no known parent package

In [50]:
# Check hdf file
from prok_datasets import IterableProkBERTPretrainingDataset, ProkBERTPretrainingHDFDataset, ProkBERTPretrainingDataset

ds = IterableProkBERTPretrainingDataset(hdf_file, input_batch_size=482, max_iteration_over_ds=2)

for i, r in enumerate(ds):
    #r[0:10]
    pass

ds = ProkBERTPretrainingHDFDataset(hdf_file)
#len(ds)
#ds[0:2]
ds = ProkBERTPretrainingDataset(X)



2023-08-14 09:38:22,001 - INFO - Dataset size: 482
2023-08-14 09:38:22,010 - INFO - Stopping the iteration.


In [ ]:
np.floor(3.1)


In [ ]:
with h5py.File(hdf_file, 'r') as dataset_file:
    dataset_file['training_data']['X'].shape[0]



In [19]:
ds = IterableProkBERTPretrainingDataset(hdf_file, input_batch_size=482, max_iteration_over_ds=2)

ds[0:1]
ds.dataset_file.close()
#ds = IterableProkBERTPretrainingDataset(hdf_file, input_batch_size=482, max_iteration_over_ds=2)

ds[0:3]




2023-08-14 09:28:11,629 - INFO - Dataset size: 482
Exception ignored in: <function IterableProkBERTPretrainingDataset.__del__ at 0x7ff3886c5a60>
Traceback (most recent call last):
  File "/home/ligeti/github/prokbert/src/prokbert/prok_datasets.py", line 125, in __del__
    if not self.dataset_file.closed:
AttributeError: 'File' object has no attribute 'closed'


[tensor([   2, 3329, 4038, 3103,  427, 2667, 1639, 1587,  753, 3784, 3130,  867,
         1514, 3671, 1331,  745, 3653, 1029,    8,   57,  838, 1045,  266,   86,
         1314,  484, 3573, 3851,  106, 1622, 1314,  478, 3482, 2396, 1411, 2036,
         3833, 3911, 1063,  556,  642, 2005, 3341,  136, 2107,  869, 1545,   72,
         1077,  774,   22,  291,  492, 3704, 1853,  910, 2199, 2354,  730, 3414,
         1301,  265,   79, 1193, 2642, 1248, 3521, 3015, 3118,  672, 2499, 3049,
         3655, 1069,  646, 2078,  419, 2537, 3665, 1228, 3196, 1918, 1941, 2316,
          123, 1902, 1697, 2514, 3291, 3443, 1772, 3714, 2013, 3468, 2178, 2012,
         3454, 1941, 2322,  228, 3586, 4060, 3454, 1942, 2325,  275,  234, 3684,
         1540, 4093, 3980, 2170, 1891, 1510, 3607,  304,  696, 2879,  947, 2804,
         3833, 3919, 1192, 2619,  872, 1598,  927, 2472, 2619,  870, 1559,  297,
          582, 1058,  478, 3487, 2472, 2619,  870, 1570,  473, 3405, 1164, 2173,
         1925, 2058,   97, 1

KeyError: 'Unable to synchronously open object (invalid identifier type to function)'

In [30]:
ds = IterableProkBERTPretrainingDataset(hdf_file, input_batch_size=482, max_iteration_over_ds=2)
ds.dataset_file.close()

ds._ensure_file_open()

ds.dataset_file.mode



2023-08-14 09:32:07,657 - INFO - Dataset size: 482


'r'

In [20]:
dataset_file = h5py.File(hdf_file, 'r')




In [5]:
dataset_file.closed


AttributeError: 'File' object has no attribute 'closed'